In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('ResultadosElectorales_2023.csv')

## Presidente afirmativos

In [ ]:
presidente_columns = ['distrito_nombre', 'seccionprovincial_nombre', 'seccion_nombre',
       'circuito_nombre', 'mesa_id', 'mesa_tipo', 'mesa_electores',
       'agrupacion_nombre', 'votos_cantidad']

In [ ]:
presidente = df[df['cargo_id']==1]

In [ ]:
presidente_afirmativos = presidente[presidente['votos_tipo']=='POSITIVO']

In [ ]:
presidente_afirmativos[presidente_columns].to_csv('presidente_afirmativos.csv', index=False)

## Presidente invalidos

In [ ]:
presidente_invalidos = presidente[presidente['votos_tipo']!='POSITIVO']

In [ ]:
presidente_invalidos[presidente_columns].to_csv('presidente_invalidos.csv', index=False)